# Circuit simulation examples

In [1]:
import obi_one as obi

In [2]:
# circuit_path_prefix = "/Users/james/Documents/obi/additional_data/O1_data/O1_data/"
circuit_path_prefix = "/Users/mwr/Documents/artefacts/SONATA/O1_data/"
# circuit_path_prefix = "/Users/pokorny/Data/Circuits/"

### Loading two circuits

In [3]:
# circuit = obi.Circuit(name="ToyCircuit-S1-6k", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit = obi.Circuit(name="O1", path=circuit_path_prefix + "circuit_config_postfix.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'O1' with 785632 neurons and 560631825 synapses


In [4]:
# circuit2 = obi.Circuit(name="ToyCircuit-S1-6k-duplicate", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
# circuit2 = obi.Circuit(name="nbS1-HEX0-beta", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit2 = obi.Circuit(name="O1_2", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes.size} neurons and {circuit2.sonata_circuit.edges.size} synapses")

Circuit 'O1_2' with 785632 neurons and 560631825 synapses


In [5]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(name="O1 Simulation", description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
node_pop = circuit._default_population_name(circuit.sonata_circuit)

sim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", node_population=node_pop)
stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=[10, 20], node_population=node_pop)
replay_neuron_set = obi.PredefinedNeuronSet(node_set="proj_Thalamocortical_POM_Source", random_sample=0.25, node_population="POm")
rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=100, node_population=node_pop)
property_neuron_set = obi.PropertyNeuronSet(
    property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": [2, 3], "synapse_class": ["INH"]}),
                     obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": [1, 2, 3], "synapse_class": ["EXC"]})],
    node_population=node_pop
)
sim_conf.add(sim_neuron_set, name='L1All')
sim_conf.add(stim_neuron_set, name='L1Stim')
sim_conf.add(rec_neuron_set, name='L1Rec')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')
sim_conf.add(replay_neuron_set, name="POM_input")

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, stim_duration=800, frequency=20, source_neuron_set=replay_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [6]:
# simulations_form.model_dump(mode="json")

In [6]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')

[2025-06-13 16:20:05,066] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-13 16:20:05,067] INFO: stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
[2025-06-13 16:20:05,067] INFO: neuron_sets.L1Stim.random_sample: [10, 20]
[2025-06-13 16:20:05,068] INFO: neuron_sets.PropertyNeuronSet.property_filter: [layer=2,3,synapse_class=INH, layer=1,2,3,synapse_class=EXC]
[2025-06-13 16:20:05,068] INFO: initialize.circuit: [Circuit(type='Circuit', name='O1', path='/Users/mwr/Documents/artefacts/SONATA/O1_data/circuit_config_postfix.json', matrix_path=None), Circuit(type='Circuit', name='O1_2', path='/Users/mwr/Documents/artefacts/SONATA/O1_data/circuit_config.json', matrix_path=None)]
[2025-06-13 16:20:05,069] INFO: 
COORDINATE PARAMETERS
[2025-06-13 16:20:05,069] INFO: stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Stim.random_sample: 10, neuron_sets.PropertyNeuronSet.property_filter: layer=2,3,synapse_class=INH, initialize.circuit: O1
[2025-06-13 16:20:05,069] INFO: stimuli.CurrentStimulus.amplitude

{0: None,
 1: None,
 2: None,
 3: None,
 4: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None,
 14: None,
 15: None}

In [15]:
# Spot check of generated spikes files
import h5py

h5 = h5py.File("../../obi-output/circuit_simulations/grid_scan/stimuli.CurrentStimulus.amplitude=0.2/neuron_sets.L1Stim.random_sample=10/neuron_sets.PropertyNeuronSet.property_filter=layer=1,2,3,synapse_class=EXC/initialize.circuit=O1/PoissonInputStimulus_spikes.h5", "r")

print(h5["spikes"].keys())
print(h5["spikes/POm"]["node_ids"][:])
h5.close()

<KeysViewHDF5 ['POm']>
[3158 1312    2 ... 4712 6054 4093]


In [7]:
# Deserialization
grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../obi-output/circuit_simulations/grid_scan/run_scan_config.json")

In [8]:
grid_scan_ds

GridScan(type='GridScan', form=SimulationsForm(type='SimulationsForm', timestamps={'stim_times': RegularTimestamps(type='RegularTimestamps', start_time=0.0, number_of_repetitions=3, interval=1000.0)}, stimuli={'CurrentStimulus': ConstantCurrentClampSomaticStimulus(type='ConstantCurrentClampSomaticStimulus', timestamps=TimestampsReference(type='TimestampsReference', block_dict_name='timestamps', block_name='stim_times'), delay=0.0, duration=5.0, neuron_set=NeuronSetReference(type='NeuronSetReference', block_dict_name='neuron_sets', block_name='L1Stim'), represents_physical_electrode=False, amplitude=[0.2, 0.5]), 'SyncCurrentStimulus': ConstantCurrentClampSomaticStimulus(type='ConstantCurrentClampSomaticStimulus', timestamps=TimestampsReference(type='TimestampsReference', block_dict_name='timestamps', block_name='stim_times'), delay=0.0, duration=100.0, neuron_set=NeuronSetReference(type='NeuronSetReference', block_dict_name='neuron_sets', block_name='L1Stim'), represents_physical_electr